## Rebuild, compile and train model_1, model_2 and model_5 using the Keras Sequential API instead of the Functional API.

In [1]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [3]:
# Unzip the data
unzip_data("nlp_getting_started.zip")

In [4]:
# Visualizing the text
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
# Let's visualize random training examples
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("-----\n")

Target: 0 (not real disaster)
Text:
POTUS appoints Brig. Gen. Richard G. Kaiser as member of the Mississippi River Commission. Learn more about the MRC: http://t.co/vdUKcV7YJy

-----

Target: 0 (not real disaster)
Text:
@myrtlegroggins &lt;gasp!&gt; I forgot Sunday! OMG

-----

Target: 0 (not real disaster)
Text:
[+]

Such a lonely day
And it's mine
It's a day that I'm glad I survived ...

??

-----

Target: 1 (real disaster)
Text:
Even though BSG had been sufficiently hyped up for me in all the years I somehow delayed watching it I was utterly utterly blown away.

-----

Target: 0 (not real disaster)
Text:
@TheEmoBrago back doing another jitsu making a hexagon on the ground as you laid there.* 64 palms of whirlwind * I yelled as air began to +

-----



In [9]:
# Splitting the training and validation sets
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)


In [10]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [11]:
# Text vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split=None,
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [12]:
# Finding the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 CANADA BC DROUGHT: Okanagan region issued Level 4 rating - Okanagan River (Columbia trib) fishing suspended to Sep 30 http://t.co/r4yZHxk7lw      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1429,  632,  377, 2829, 1486,  886,  813,  178, 4798, 2829,  499,
        3988, 7376, 3846, 7891]], dtype=int64)>

In [16]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [17]:
# Creating the embedding layer
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)

embedding

In [18]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Falling asleep to the sounds to thousands of River Plate fans in the stadium and a thunderstorm. #VivaArgentina      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04604534,  0.01367277, -0.04481597, ..., -0.00725137,
          0.00843942, -0.04973448],
        [-0.02762867, -0.01608794, -0.02418238, ...,  0.01570188,
          0.01355514, -0.03290309],
        [ 0.03489255,  0.01067371,  0.01911645, ...,  0.02135624,
         -0.02035036, -0.01717801],
        ...,
        [-0.03642768, -0.0496197 , -0.01994752, ..., -0.02752961,
          0.04322295,  0.00236944],
        [-0.0327631 , -0.00551457,  0.01429594, ...,  0.02609319,
         -0.04378529,  0.02386046],
        [ 0.00100335,  0.02237889,  0.02475416, ..., -0.00936385,
         -0.00569763, -0.03894851]]], dtype=float32)>

### Rebuild, compile and train model_1, model_2 and model_5 using the Keras Sequential API instead of the Functional API.

### Making the baseline model using Naive bays

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenizatin and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [20]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model acheives an accuracy of: {baseline_score*100:.2f}%")

Our baseline model acheives an accuracy of: 79.27%


In [23]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [21]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall, f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [24]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Food Forward Neural Network

In [25]:
from tensorflow.keras import Sequential, layers

model_1 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    embedding,
    layers.GlobalAveragePooling1D(),
    layers.Dense(1, activation="sigmoid")
], name="model_1_dense")

In [26]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))


Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.6125 - accuracy: 0.6869 - val_loss: 0.5363 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4415 - accuracy: 0.8210 - val_loss: 0.4722 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3470 - accuracy: 0.8610 - val_loss: 0.4561 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2844 - accuracy: 0.8914 - val_loss: 0.4636 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2379 - accuracy: 0.9133 - val_loss: 0.4790 - val_accuracy: 0.7874


In [28]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4790 - accuracy: 0.7874


[0.47895196080207825, 0.787401556968689]

In [29]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

24/24 [==============================] - 0s 5ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [30]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7927656871284042,
 'recall': 0.7874015748031497,
 'f1': 0.7842639469124086}

In [31]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

Model 2: LSTM

In [39]:
model_2 = tf.keras.Sequential([
    layers.Input(shape=(1,), dtype="string"),
    text_vectorizer,
    embedding,
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(96),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_2_LSTM")

In [40]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_3 (LSTM)               (None, 15, 128)           131584    
                                                                 
 lstm_4 (LSTM)               (None, 15, 128)           131584    
                                                                 
 lstm_5 (LSTM)               (None, 96)                86400     
                                                                 
 dense_2 (Dense)             (None, 64)                6208      
                                                      

In [41]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_2.history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 15s 36ms/step - loss: 0.1543 - accuracy: 0.9494 - val_loss: 0.7025 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1013 - accuracy: 0.9588 - val_loss: 1.1768 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0845 - accuracy: 0.9657 - val_loss: 1.0427 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0675 - accuracy: 0.9696 - val_loss: 1.3469 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0602 - accuracy: 0.9723 - val_loss: 1.3240 - val_accuracy: 0.7585


In [42]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

24/24 [==============================] - 2s 17ms/step


In [43]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.8530183727034,
 'precision': 0.7581771522804598,
 'recall': 0.7585301837270341,
 'f1': 0.7577141851857451}